##### We will prepare the dataset for the prognosis part of the AML.
##### To do so, we will need :
##### -ELN data : We have followed Yoann's methodology and change the ELN classification coming from the Master dataset
##### -Clinical data : it comes frome Master Dataset
##### -Demographic data: it comes from Master dataset
##### -Genetical data with ITD : ITD has been added based on rules we have established looking at the distribution plots of NGS and Clinical ITD and number of read counts. Rules are based on the provenance of ITD
##### (wheter it comes from clinical or NGS or both and the standard deviation of the read counts taking into account the high variability of read counts and its impact on ITD).
##### -Cytogenetical data: it has already been modified based on the rules established with Elli (look at frequency for additions and deletions and keep the ones greater than 2%)
##### -Translocation data: it has been modified based on Elli's rules. We keep only translocation with a count number greater than or equal to 2 . We summarize other translocations
##### that were not kept in a column called other translocs
##### -Column complex to integrate to cytogenetical data: we have modified this column based on some mistakes for complex classification previously done. The rules adopted are the presence of 3 or more aneuploidies 
##### (additions or deletions combined and no presence of translocation(8,21) or (15,17)
##### -Component data: it comes from the component that we have established thanks to the HDP method.


In [2]:
library('hdp')
library('clusterCrit')
library('grid')
library('gridExtra')
library('ggplot2')
library('ggrepel')
library('RColorBrewer')
library('dplyr')
library('reshape2')
library('IRdisplay')
library('scales')
library('survival')
library('corrplot')
library('Hmisc')
source('../../../src/tools.R')     # custom tools function
source('../../../src/hdp_tools_yanis.R')
###       
source("../../../src/merge_df.R")
source("../../../src/my_toolbox.R")
source("../../../src/my_components.R")
source("../../../src/my_utils.R")
source("../../../src/ggstyles.R")
source("../../../src/my_hotspots.R")
###


theme_set(theme_minimal())

# set jupyer notebook parameters
options(repr.plot.res        = 100, # set a medium-definition resolution for the jupyter notebooks plots (DPI)
        repr.matrix.max.rows = 200, # set the maximum number of rows displayed
        repr.matrix.max.cols = 200) # set the maximum number of columns displayed

##### In the dataframe already prepared,(df_all_components), there is already the genetical data WITH ITD , the translocations, the cytogenetical data and the components.
##### We also have demographic and clinical data from the Master dataset.
##### We need to add:
##### -ELN coming from another dataset that I have set up for the new rule
##### -complex: we will add a new column called complex_new


In [3]:
df_all_components <- read.table("../../../data/updated_dataset/all_components.tsv",sep = '\t' , header = T)# 
df_initial <- read.table("../../../data/initial_dataset/Master_04_10_2019.csv",sep = ',' , header = T)
rownames(df_initial) <- df_initial$data_pd
df_initial <- df_initial[,-1:-4]


In [4]:
df_eln <- read.table("../../../data/updated_dataset/modif_final_eln.csv",sep = ',' , header = T)
rownames(df_eln) <- df_eln$data_pd
df_eln <- df_eln[,-1:-2]
names(df_eln)[names(df_eln) == "eln_2017"] <- "new_eln"

In [5]:
df_all <- merge(df_eln['new_eln'],df_all_components,by=0)
rownames(df_all) <- df_all$Row.names
df_all <- df_all[,-1]
# now we have genetical with ITD ,  translocation , cytogenetical , components and eln
# we reorder inv_3 to have the columns in a nice order: eln,genetical,cytogenetical,translocationm and component
df_all <- df_all[,c(1,seq(3,ncol(df_all)-1),2,ncol(df_all))]

##### Now we use df_initial for clinical ,  demographical and new rules for complex:
##### Let's first do the new rules for complex

In [6]:
for (i in 1:22){
    df_initial[,paste("add_",as.character(i),sep="")] <- df_initial[,paste("add_","p",sep=as.character(i))]+df_initial[,paste("add_","q",sep=as.character(i))]+df_initial[,paste("plus",as.character(i),sep="")]
    df_initial[,paste("add_",as.character(i),sep="")][df_initial[,paste("add_",as.character(i),sep="")]>=2] <-1
} 
df_initial$add_x <- df_initial$add_xp + df_initial$add_xq + df_initial$plusx
df_initial$add_y <- df_initial$plusy
###
# deletions
###
for (i in c(c(1,2,3),5:13,15:19)){
    df_initial[,paste("del_",as.character(i),sep="")] <- df_initial[,paste("del_","p",sep=as.character(i))]+df_initial[,paste("del_","q",sep=as.character(i))]+df_initial[,paste("minus",as.character(i),sep="")]
    df_initial[,paste("del_",as.character(i),sep="")][df_initial[,paste("del_",as.character(i),sep="")]>=2] <-1  
}
for (i in c(c(4,14,20,21,22),"x")){
    df_initial[,paste("del_",as.character(i),sep="")]<- df_initial[,paste("del_","q",sep=as.character(i))]+df_initial[,paste("minus",as.character(i),sep="")]
}     
df_initial$del_y <- df_initial$minusy
df_initial$sum <- rowSums(df_initial[,519:566],na.rm=T)
df_initial$new_complex <- ifelse((df_initial$sum>=3) & (df_initial$t_8_21==0) & (df_initial$t_15_17==0), 1,0) 

##### Let's now add clinical and demographical to merge them all together:

In [7]:
clinical <-c("ahd","perf_status","bm_blasts","secondary","wbc","hb","plt")
demographical <- c("gender","age")
survival <- c("os","os_status","cr")

In [8]:
df_all <- merge(df_all,df_initial[,c(clinical,demographical,survival,"new_complex")],by=0)
rownames(df_all) <- df_all$Row.names
df_all <- df_all[,-1]

In [8]:
df_all$complex <- NULL
dim(df_all)

[1] 2094  166

In [20]:
row.has.na <- apply(df_all, 1, function(x){any(is.na(x))})

In [25]:
table(df_all$os_status)


   0    1 
 732 1342 

In [24]:
dim(df_all)
dim(na.omit(df_all))

[1] 2094  166

[1] 1933  166

In [ ]:
# Prep without comp
correlates <- c("ahd","perf_status","bm_blasts","secondary","wbc","hb","plt","gender","age","os","os_status")

df_all_components <- read.table("../../../data/updated_dataset/refined_components_updated.tsv",sep = '\t' , header = T)
df_initial <- read.table("../../../data/initial_dataset/Master_04_10_2019.csv",sep = ',' , header = T)
rownames(df_initial) <- df_initial$data_pd
df_initial <- df_initial[,-1:-4]
df_eln <- read.table("../../../data/updated_dataset/eln_ratio.tsv",sep = '\t' , header = T)
rownames(df_eln) <- df_eln$X
df_eln <- df_eln[-1]
df <- merge(df_eln,df_all_components,by=0)
rownames(df) <- df$Row.names
df <- df[-1]
df <- merge(df,df_initial[,correlates],by=0)
rownames(df) <- df$Row.names
df <- df[-1]
df <- df[,c(1:151,157,156,166:176)]
df <- na.omit(df)
df <- df[df$os>0,]
df$eln_2017_ratio <- ifelse(df$eln_2017_ratio=="adverse",1,
                     ifelse(df$eln_2017_ratio=="intermediate",2,3))
write.table(df,"prognosis_without_comp.tsv",sep='\t')

In [1]:
dim(read.table("../../../data/updated_dataset/refined_components_updated.tsv",sep = '\t' , header = T))

[1] 2094  164

In [19]:
# Prep with component Elli
library(dummies)
df_component_yanis <- read.table("../../../data/updated_dataset/refined_components_updated.tsv",sep = '\t' , header = T)
df_component_elli <- read.table("../../../data/updated_dataset/elli_refined_components_updated.tsv",sep = '\t' , header = T)
df <- read.table("prognosis_without_comp.tsv",sep='\t')
df_yanis <- merge(df,df_component_yanis[,'final_component',drop=FALSE],by=0)
df_elli <- merge(df,df_component_elli[,'final_component',drop=FALSE],by=0)
rownames(df_yanis) <- df_yanis$Row.names
rownames(df_elli) <- df_elli$Row.names
df_yanis <- df_yanis[-1]
df_elli <- df_elli[-1]
df_yanis <- cbind(df_yanis, dummy('final_component',df_yanis, sep = "_"))
df_elli <- cbind(df_elli, dummy('final_component',df_elli, sep = "_"))
df_yanis <- df_yanis[,c(1:162,166:184,163,164,165)]
df_elli <- df_elli[,c(1:162,166:184,163,164,165)]
#write.table(df_yanis,"prognosis_comp_yanis.tsv",sep='\t')
#write.table(df_elli,"prognosis_comp_elli.tsv",sep='\t')

In [31]:
# Prep final component
df_component_final <- read.table("../../../data/updated_dataset/final_refined_components_updated.tsv",sep = '\t' , header = T)
df <- read.table("prognosis_without_comp.tsv",sep='\t')
df_final <- merge(df[,colnames(df)!="U2AF1"],df_component_final[c("U2AF1_p.S34","U2AF1_p.Q157","final_component")],by=0)
rownames(df_final) <- df_final$Row.names
df_final <- df_final[-1]
df_final <- cbind(df_final, dummy('final_component',df_final, sep = "_"))
df_final <- df_final[,c(1:81,164,165,82:161,167:183,162,163,166)]
write.table(df_final,"prognosis_comp_final.tsv",sep='\t')

In [2]:
dim(read.table("prognosis_comp_final.tsv"))

[1] 1931  183

In [3]:
df_component_final <- read.table("../../../data/updated_dataset/final_refined_components_updated.tsv",sep = '\t' , header = T)
dim(df_component_final)

[1] 2094  165

In [9]:
correlates <- c("ahd","perf_status","bm_blasts","secondary","wbc","hb","plt","gender","age","os","os_status")
df_all_components <- read.table("../../../data/updated_dataset/refined_components_updated.tsv",sep = '\t' , header = T)
df_initial <- read.table("../../../data/initial_dataset/Master_04_10_2019.csv",sep = ',' , header = T)
rownames(df_initial) <- df_initial$data_pd
df_initial <- df_initial[,-1:-4]
df_eln <- read.table("../../../data/updated_dataset/eln_ratio.tsv",sep = '\t' , header = T)
rownames(df_eln) <- df_eln$X
df_eln <- df_eln[-1]
df <- merge(df_eln,df_all_components,by=0)
rownames(df) <- df$Row.names
df <- df[-1]
df <- merge(df,df_initial[,correlates],by=0)
rownames(df) <- df$Row.names

In [12]:
 df[,names(df)[!!colSums(is.na(df))]]

,ahd,perf_status,bm_blasts,secondary,wbc,hb,plt,gender,age,os,os_status
PD14868a,1,1,89.00000,1,175.0,9.899994,199,0,79.71253,0.19164956,1
PD14869c,1,1,90.00000,1,3.1,8.699997,23,1,65.04312,4.70636550,0
PD14871a,1,0,62.00000,1,127.0,7.599998,57,1,79.65777,0.37782341,1
PD14872a,1,2,40.00000,1,8.2,7.699997,124,0,66.50513,0.04928131,1
PD14873a,0,1,25.00000,2,44.2,9.299995,40,1,71.78097,0.55030801,1
PD14874a,1,0,56.00000,1,69.0,9.099998,63,1,75.01437,0.20807666,1
PD14875a,1,2,70.00000,1,43.4,10.299995,64,0,74.79535,1.58795346,1
PD14876c,0,1,90.00000,2,59.1,7.599998,104,1,65.65914,0.24366872,1
PD14877c,1,1,92.00000,1,144.0,6.299999,47,0,84.10404,0.06570842,1
PD14879a,1,0,84.00000,1,4.6,10.599998,25,1,76.50103,0.11225188,1
